In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import requests

In [ ]:
#DOWNLOAD ECONOMIC CALENDAR DATA GMT +2
url = 'https://economic-calendar.tradingview.com/events'
today = pd.Timestamp.today().normalize().date()
payload = {
    'from': (today - pd.offsets.Day(14)).date().isoformat(),
    'to': today.isoformat(),
    'countries': ','.join(['US', 'IN'])
}
data = requests.get(url, params=payload).json()
df = pd.DataFrame(data['result'])

In [ ]:
def download_data(cur_1, cur_2, t_period='7d', t_interval='1m'):
    #DOWNLOAD DATA
    data = yf.Ticker(f'{cur_1}{cur_2}=X')
    f_data = data.history(period=t_period, interval=t_interval)
    #REMOVE TABLES WITH 0
    mask = f_data.ne(0).any(axis=0)
    f_data = f_data.loc[:, mask]
    #RENAME COLUMNS
    f_data.rename(columns={'Open':f'{cur_1}{cur_2}_OPEN', 'High':f'{cur_1}{cur_2}_HIGH', 
                         'Low':f'{cur_1}{cur_2}_LOW', 'Close':f'{cur_1}{cur_2}_CLOSE'}, inplace=True)
    return f_data

In [ ]:
#LOAD OLD DATA
old_data = pd.read_excel('/kaggle/input/forex-data-gatherer/FOREX_DATA.xlsx')
old_data.set_index('Datetime', inplace=True)

#CURRENCIES PAIRS WHICH WE WANT DOWNLOAD
main_cur = ['PLN', 'EUR']
additional_cur = ['CZK', 'HUF', 'USD', 'CHF', 'GBP', 'JPY']

#DOWNLOAD OUR MAIN CURRENCY PAIR
forex_data = download_data('EUR', 'PLN')

#DOWNLOAD OUR ADDITIONAL CURRENCY PAIRS
for main in main_cur:
    for add in additional_cur:
        #DOWNLOAD PAIR DATA
        temp_data = download_data(main, add)
        #JOIN TO MAIN TABLE
        forex_data = forex_data.join(temp_data)
        
#CHANGE DATETIME TO STRING IN ORDER TO SAVE TO XLSX
forex_data.index = forex_data.index.strftime('%d-%m-%Y %H:%M:%S %z')

#JOIN OLD AND NEW DATA(FROM THIS WEEK)
forex_data = pd.concat([forex_data, old_data])

#REMOVE DUPLICATES
forex_data.reset_index(inplace=True)
forex_data.drop_duplicates(['Datetime'], inplace=True)
forex_data.set_index('Datetime', inplace=True)

#SAVE TO EXCEL
forex_data.to_excel('FOREX_DATA.xlsx')